# Documentation for call methods

https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.

https://python.langchain.com/docs/modules/chains/how_to/call_methods

In [10]:
import openai

from dotenv import load_dotenv, find_dotenv, dotenv_values

import sys
sys.path.append('../')

import utils.data
import utils.models
import utils.prompting

import pprint as pp

In [2]:
_ = load_dotenv(find_dotenv())
if _ is not True:
    raise Exception("No .env file found")

In [3]:
data = utils.data.load_data()
data = utils.data.flatten_CoQA(data)
len(data)

10

In [5]:
prompt_template = utils.prompting.create_template(
    prompting_type='baseline_SBS',
    dataset='commonsenseQA',
    output_formatting=False)

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

llm = ChatOpenAI(temperature = 0)

In [7]:
chain = utils.models.create_LLMchain(llm, prompt_template, verbose=True)

## Use LLMChain; format prompt during the call
+ current call in main.py
+ only the first instance gets an explanation, the others just the answer (!! looks like it's not just the firs that gets an explanation, but other ones as well when running on more instances..)
+ prompt formatting is okay (all prompts contains 'lets think sbs')

In [18]:
import pprint as pp

responses = {}
for idx, instance in enumerate(data):
    response = chain(
        return_only_outputs=True, 
        inputs={
            'question':instance["stem"],
            'choice_A':instance["choice_A"],
            'choice_B':instance["choice_B"],
            'choice_C':instance["choice_C"],
            'choice_D':instance["choice_D"],
            'choice_E':instance["choice_E"]
            }
        )
    responses[idx] = response['text']

print(f"Responses:\n{pp.pformat(responses)}\n")



> Entering new LLMChain chain...
Prompt after formatting:
Human: Answer the following question. There are 5 possible answer and only 1 is correct. 

Question: A revolving door is convenient for two direction travel, but it also serves as a security measure at a what? 

- A: bank 
- B: library 
- C: department store 
- D: mall 
- E: new york 

Choose the correct answer. Let's think step by step.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Answer the following question. There are 5 possible answer and only 1 is correct. 

Question: What do people aim to do at work? 

- A: complete job 
- B: learn from each other 
- C: kill animals 
- D: wear hats 
- E: talk to each other 

Choose the correct answer. Let's think step by step.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Answer the following question. There are 5 possible answer and only 1 is correct. 

Question: Where would you find magazines along side m

## No Chain; format prompt before the call
+ same problem

In [8]:
prompts = []

for instance in data:
    prompt_lc = prompt_template.format_messages(
        question=instance["stem"],
        choice_A=instance["choice_A"],
        choice_B=instance["choice_B"],
        choice_C=instance["choice_C"],
        choice_D=instance["choice_D"],
        choice_E=instance["choice_E"]
    )
    prompts.append(prompt_lc)

prompts

[[HumanMessage(content="Answer the following question. There are 5 possible answer and only 1 is correct. \n\nQuestion: A revolving door is convenient for two direction travel, but it also serves as a security measure at a what? \n\n- A: bank \n- B: library \n- C: department store \n- D: mall \n- E: new york \n\nChoose the correct answer. Let's think step by step.", additional_kwargs={}, example=False)],
 [HumanMessage(content="Answer the following question. There are 5 possible answer and only 1 is correct. \n\nQuestion: What do people aim to do at work? \n\n- A: complete job \n- B: learn from each other \n- C: kill animals \n- D: wear hats \n- E: talk to each other \n\nChoose the correct answer. Let's think step by step.", additional_kwargs={}, example=False)],
 [HumanMessage(content="Answer the following question. There are 5 possible answer and only 1 is correct. \n\nQuestion: Where would you find magazines along side many other printed works? \n\n- A: doctor \n- B: bookstore \n- C

In [11]:
responses = {}
for idx, prompt in enumerate(prompts):
    response = llm(prompt)
    responses[idx] = response.content

print(f"Responses:\n{pp.pformat(responses)}\n")

Responses:
{0: 'The question asks about a revolving door serving as a security measure. '
    'To determine the correct answer, we need to consider which location would '
    'benefit from having a security measure like a revolving door.\n'
    '\n'
    'A: A bank is a possible option as it often has security measures in '
    'place. A revolving door could help control access and prevent '
    'unauthorized entry.\n'
    '\n'
    'B: A library may not require a revolving door as a security measure. It '
    'is more likely to focus on other security measures such as surveillance '
    'cameras or alarms.\n'
    '\n'
    'C: A department store could benefit from a revolving door as a security '
    'measure to control the flow of customers and deter theft.\n'
    '\n'
    'D: A mall is another possible option as it often has multiple entrances '
    'and exits. A revolving door could help regulate the flow of people and '
    'enhance security.\n'
    '\n'
    'E: "New York" is not a s

## Isolated calls (no loop)
+ same problem

In [24]:
response = llm(prompts[1])
response.content

'The correct answer is B: learn from each other.'

In [25]:
response = llm(prompts[2])
response.content

'The correct answer is B: bookstore. Bookstores typically have a wide range of printed works, including magazines, available for purchase.'

## Format prompt for Chat model (sys message, human message separate)
+ formatting the prompt according to chat models guidelines does not solve the problem. The generation is exactly the same.
+ !! changed the sys prompt and solved the issue. Therefore the issue is with the prompt (let's think sbs is not enough to generate explanations with this model)

In [12]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [26]:
sys_message_prompt = SystemMessagePromptTemplate.from_template(
    "Your task is to answer the question. There are 5 possible answer and only one is correct. Always choose exactly one answer. Always explain your answer. Think step by step."
)

In [14]:
human_message_prompt = HumanMessagePromptTemplate.from_template(
    "Question: {question}\n\nChoices:\n-A: {choice_A}\n-B: {choice_B}\n-C: {choice_C}\n-D: {choice_D}\n-E: {choice_E}\n\n"
)

In [27]:
chat_prompt_template = ChatPromptTemplate.from_messages([sys_message_prompt, human_message_prompt])

In [30]:
chat_prompt_template

ChatPromptTemplate(input_variables=['choice_A', 'choice_B', 'choice_C', 'choice_D', 'choice_E', 'question'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='Your task is to answer the question. There are 5 possible answer and only one is correct. Always choose exactly one answer. Always explain your answer. Think step by step.', template_format='f-string', validate_template=True), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['choice_A', 'choice_B', 'choice_C', 'choice_D', 'choice_E', 'question'], output_parser=None, partial_variables={}, template='Question: {question}\n\nChoices:\n-A: {choice_A}\n-B: {choice_B}\n-C: {choice_C}\n-D: {choice_D}\n-E: {choice_E}\n\n', template_format='f-string', validate_template=True), additional_kwargs={})])

In [28]:
chat_prompts = []
for instance in data:
    chat_prompt = chat_prompt_template.format_prompt(
        question=instance["stem"],
        choice_A=instance["choice_A"],
        choice_B=instance["choice_B"],
        choice_C=instance["choice_C"],
        choice_D=instance["choice_D"],
        choice_E=instance["choice_E"]
    ).to_messages()
    chat_prompts.append(chat_prompt)

chat_prompts

[[SystemMessage(content='Your task is to answer the question. There are 5 possible answer and only one is correct. Always choose exactly one answer. Always explain your answer. Think step by step.', additional_kwargs={}),
  HumanMessage(content='Question: A revolving door is convenient for two direction travel, but it also serves as a security measure at a what?\n\nChoices:\n-A: bank\n-B: library\n-C: department store\n-D: mall\n-E: new york\n\n', additional_kwargs={}, example=False)],
 [SystemMessage(content='Your task is to answer the question. There are 5 possible answer and only one is correct. Always choose exactly one answer. Always explain your answer. Think step by step.', additional_kwargs={}),
  HumanMessage(content='Question: What do people aim to do at work?\n\nChoices:\n-A: complete job\n-B: learn from each other\n-C: kill animals\n-D: wear hats\n-E: talk to each other\n\n', additional_kwargs={}, example=False)],
 [SystemMessage(content='Your task is to answer the question

In [29]:
response = llm(chat_prompts[1])
response

AIMessage(content="The correct answer is A: complete job.\n\nWhen people go to work, their primary aim is to complete the tasks and responsibilities assigned to them. This involves performing their job duties, meeting deadlines, and achieving the goals set by their employers. The purpose of work is to contribute to the success of the organization or company they work for, whether it's through producing goods, providing services, or fulfilling specific roles within the company's structure. While learning from each other, wearing hats, and talking to each other may occur in a work environment, they are not the primary objectives of work. Killing animals is not a typical aim of work, unless someone is specifically employed in a profession related to hunting or animal control.", additional_kwargs={}, example=False)

## Change prompt so that it's only the question + let's think sbs
### + output formatting

In [52]:
template_str = """
Question: {question}\n\n
Choices:\n-A: {choice_A}\n-B: {choice_B}\n-C: {choice_C}\n-D: {choice_D}\n-E: {choice_E}\n\n
Let's think step by step.\n\n
{format_instructions}
"""

new_prompt_template = ChatPromptTemplate.from_template(template_str)

In [53]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

prediction_letter_schema = ResponseSchema(
    name="prediction_letter", 
    description="This is the answer to the question. It is one letter (either A, B, C, D, or E), which corresponds to the answer chosen by the model."
)

prediction_text_schema = ResponseSchema(
    name="prediction_text", 
    description="This is the answer to the question. It is the text corresponding to the letter chosen as the answer. It is usually one word or a short phrase."
)

explanation_schema = ResponseSchema(
    name="explanation", 
    description="This is the explanation for the prediction. It is a piece of text explains why the model chose the answer it did."
)

schemas = [prediction_letter_schema, prediction_text_schema, explanation_schema]

In [54]:
output_parser = StructuredOutputParser.from_response_schemas(schemas)

In [55]:
format_instructions = output_parser.get_format_instructions(only_json=True)
format_instructions

'\n```json\n{\n\t"prediction_letter": string  // This is the answer to the question. It is one letter (either A, B, C, D, or E), which corresponds to the answer chosen by the model.\n\t"prediction_text": string  // This is the answer to the question. It is the text corresponding to the letter chosen as the answer. It is usually one word or a short phrase.\n\t"explanation": string  // This is the explanation for the prediction. It is a piece of text explains why the model chose the answer it did.\n}\n'

In [56]:
prompts = []

for instance in data:
    prompt_lc = new_prompt_template.format_messages(
        question=instance["stem"],
        choice_A=instance["choice_A"],
        choice_B=instance["choice_B"],
        choice_C=instance["choice_C"],
        choice_D=instance["choice_D"],
        choice_E=instance["choice_E"],
        format_instructions=format_instructions
    )
    prompts.append(prompt_lc)

prompts

[[HumanMessage(content='\nQuestion: A revolving door is convenient for two direction travel, but it also serves as a security measure at a what?\n\n\nChoices:\n-A: bank\n-B: library\n-C: department store\n-D: mall\n-E: new york\n\n\nLet\'s think step by step.\n\n\n\n```json\n{\n\t"prediction_letter": string  // This is the answer to the question. It is one letter (either A, B, C, D, or E), which corresponds to the answer chosen by the model.\n\t"prediction_text": string  // This is the answer to the question. It is the text corresponding to the letter chosen as the answer. It is usually one word or a short phrase.\n\t"explanation": string  // This is the explanation for the prediction. It is a piece of text explains why the model chose the answer it did.\n}\n\n', additional_kwargs={}, example=False)],
 [HumanMessage(content='\nQuestion: What do people aim to do at work?\n\n\nChoices:\n-A: complete job\n-B: learn from each other\n-C: kill animals\n-D: wear hats\n-E: talk to each other\n

In [57]:
response = llm(prompts[0])
response.content

'{\n\t"prediction_letter": "A",\n\t"prediction_text": "bank",\n\t"explanation": "A revolving door is often used as a security measure at a bank to control the flow of people entering and exiting the building. It helps prevent unauthorized access and provides a controlled entry point for security purposes."\n}'

In [58]:
responses = {}

for idx, prompt in enumerate(prompts):
    response = llm(prompt)
    responses[idx] = response.content

print(f"Responses:\n{pp.pformat(responses)}\n")


Responses:
{0: '{\n'
    '\t"prediction_letter": "A",\n'
    '\t"prediction_text": "bank",\n'
    '\t"explanation": "A revolving door is often used as a security measure at '
    'a bank to control the flow of people entering and exiting the building. '
    'It helps prevent unauthorized access and provides a controlled entry '
    'point for security purposes."\n'
    '}',
 1: '{\n'
    '\t"prediction_letter": "A",\n'
    '\t"prediction_text": "complete job",\n'
    '\t"explanation": "People aim to complete their job tasks and '
    'responsibilities at work. This includes meeting deadlines, achieving '
    'goals, and delivering results."\n'
    '}',
 2: '{\n'
    '\t"prediction_letter": "B",\n'
    '\t"prediction_text": "bookstore",\n'
    '\t"explanation": "Magazines are commonly found in bookstores, where they '
    'are displayed alongside many other printed works such as books and '
    'newspapers."\n'
    '}',
 3: '{\n'
    '\t"prediction_letter": "A",\n'
    '\t"prediction_te

### change explanation schema

- the output formatting instructions affect the generation - probably too much.. parse output in a different call? or using a different method? 
- maybe I can use another model to only extract the letter (which is what I need for evaluation of predictions) and treat the full generation as the explanation.

In [66]:
explanation_schema = ResponseSchema(
    name="full_generation", 
    description="This is the full text generated by the model."
)

schemas = [prediction_letter_schema, prediction_text_schema, explanation_schema]

output_parser = StructuredOutputParser.from_response_schemas(schemas)

format_instructions = output_parser.get_format_instructions(only_json=True)
format_instructions

'\n```json\n{\n\t"prediction_letter": string  // This is the answer to the question. It is one letter (either A, B, C, D, or E), which corresponds to the answer chosen by the model.\n\t"prediction_text": string  // This is the answer to the question. It is the text corresponding to the letter chosen as the answer. It is usually one word or a short phrase.\n\t"full_generation": string  // This is the full text generated by the model.\n}\n'

In [67]:
prompts = []

for instance in data:
    prompt_lc = new_prompt_template.format_messages(
        question=instance["stem"],
        choice_A=instance["choice_A"],
        choice_B=instance["choice_B"],
        choice_C=instance["choice_C"],
        choice_D=instance["choice_D"],
        choice_E=instance["choice_E"],
        format_instructions=format_instructions
    )
    prompts.append(prompt_lc)

prompts

[[HumanMessage(content='\nQuestion: A revolving door is convenient for two direction travel, but it also serves as a security measure at a what?\n\n\nChoices:\n-A: bank\n-B: library\n-C: department store\n-D: mall\n-E: new york\n\n\nLet\'s think step by step.\n\n\n\n```json\n{\n\t"prediction_letter": string  // This is the answer to the question. It is one letter (either A, B, C, D, or E), which corresponds to the answer chosen by the model.\n\t"prediction_text": string  // This is the answer to the question. It is the text corresponding to the letter chosen as the answer. It is usually one word or a short phrase.\n\t"full_generation": string  // This is the full text generated by the model.\n}\n\n', additional_kwargs={}, example=False)],
 [HumanMessage(content='\nQuestion: What do people aim to do at work?\n\n\nChoices:\n-A: complete job\n-B: learn from each other\n-C: kill animals\n-D: wear hats\n-E: talk to each other\n\n\nLet\'s think step by step.\n\n\n\n```json\n{\n\t"prediction_

In [68]:
responses = {}

for idx, prompt in enumerate(prompts):
    if idx < 5:
        response = llm(prompt)
        responses[idx] = output_parser.parse(response.content)
responses

{0: {'prediction_letter': 'A',
  'prediction_text': 'bank',
  'full_generation': 'A revolving door is convenient for two direction travel, but it also serves as a security measure at a bank.'},
 1: {'prediction_letter': 'A',
  'prediction_text': 'complete job',
  'full_generation': 'People aim to complete their job at work. This involves fulfilling their responsibilities, meeting deadlines, and achieving the tasks assigned to them.'},
 2: {'prediction_letter': 'B',
  'prediction_text': 'bookstore',
  'full_generation': 'The answer is B: bookstore. Magazines can be found in bookstores along with many other printed works.'},
 3: {'prediction_letter': 'A',
  'prediction_text': 'fast food restaurant',
  'full_generation': 'The most likely place to find a hamburger is at a fast food restaurant.'},
 4: {'prediction_letter': 'D',
  'prediction_text': 'farming areas',
  'full_generation': 'James should look for farmland in farming areas. These areas are specifically designated for agriculture 